<a href="https://colab.research.google.com/github/DCI-alxogm/me2025-clase-joseemiliosierra123/blob/main/Tareas/Nov_24_Ajuste_minimos_cuadrados/Ajuste_minimos_cuadrados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El código se divide en 6 secciones, a continuación se detalla cada una:

1. Se importa la librería de numpy para las funciones logaritmo y sumatoria, así como la función minimize.

2. Se importa la librería pandas para extraer y manipular fácilmente los valores del archivo de excel. Es importante aclarar que es mi primera vez utilizando estas herraminetas, así que para entender como funcionaba y cómo trata los datos, recurrí a las asistencias de gemini, lo que me permitió entender como se extraen y almacenan los valores.

3. Como se explicó en el punto 2, en esta sección se almacenan los valores del archivo de excel, clasificándolos en el tipo de grano de cafe, la temperatura y la variable en cuestión (aw: adsorción de agua, wb: datos en húmedo, db: datos en seco).

4. Se definen las funciones peleg y dlp detalladas en el artículo proporcionado, así como la ecuación de chi cuadrada.

5. Aquí, a partir de suposiciones iniciales (initial_guess), se hace uso de la función minimize para ambos métodos.

6. Finalmente se ejecuta el código, para cada temperatura, y se imprimen los valores de chi cuadrada para cada método y cada temperatura, con un total de 6 resultados.

In [83]:
#1

import numpy as np
from scipy.optimize import minimize

#2

import pandas as pd
#Extracción de datos del excel
df = pd.read_excel('datos_1.xlsx', header=None)

#3

def extract_all_coffee_data(df):
    """
    Extrae todos los datos del archivo Excel y los organiza en un diccionario
    """
    coffee_data = {
        'whole_beans': {
            25: {'aw': [], 'wb': [], 'db': []},
            35: {'aw': [], 'wb': [], 'db': []},
            45: {'aw': [], 'wb': [], 'db': []}
        },
        'fine_ground': {
            25: {'aw': [], 'wb': [], 'db': []},
            35: {'aw': [], 'wb': [], 'db': []},
            45: {'aw': [], 'wb': [], 'db': []}
        },
        'medium_ground': {
            25: {'aw': [], 'wb': [], 'db': []},
            35: {'aw': [], 'wb': [], 'db': []},
            45: {'aw': [], 'wb': [], 'db': []}
        }
    }

    # Mapeo de columnas
    column_mapping = {
        'whole_beans': {
            25: (0, 1, 2),
            35: (3, 4, 5),
            45: (6, 7, 8)
        },
        'fine_ground': {
            25: (10, 11, 12),
            35: (13, 14, 15),
            45: (16, 17, 18)
        },
        'medium_ground': {
            25: (20, 21, 22),
            35: (23, 24, 25),
            45: (26, 27, 28)
        }
    }

    # Extraer datos para cada tipo y temperatura
    for coffee_type in column_mapping:
        for temp in column_mapping[coffee_type]:
            aw_col, wb_col, db_col = column_mapping[coffee_type][temp]


            aw_data = df.iloc[0:, aw_col].astype(float)
            wb_data = df.iloc[0:, wb_col].astype(float)
            db_data = df.iloc[0:, db_col].astype(float)

            # Limpiar datos
            mask = ~(aw_data.isna() | db_data.isna())
            aw_clean = aw_data[mask].values
            wb_clean = wb_data[mask].values
            db_clean = db_data[mask].values

            coffee_data[coffee_type][temp]['aw'] = aw_clean
            coffee_data[coffee_type][temp]['wb'] = wb_clean
            coffee_data[coffee_type][temp]['db'] = db_clean


    return coffee_data

#4

def modelo_peleg(params, a_w, db_data):
  b0, b1, b2, b3 = params
  return b0 * a_w ** b1 + b2 * a_w ** b3

def modelo_dlp(params, a_w, db_data):
  b0, b1, b2, b3 = params
  x = np.log( - np.log( a_w ) )
  return b0 + b1 * x + b2 * x ** 2 + b3 * x ** 3

def chi_cuadrada_peleg(params, a_w, db_data):
  datos_experimentales = np.array(db_data)
  datos_predichos = modelo_peleg(params, a_w, db_data)
  return np.sum((datos_experimentales - datos_predichos)**2)

def chi_cuadrada_dlp(params, a_w, db_data):
  datos_experimentales = np.array(db_data)
  datos_predichos = modelo_dlp(params, a_w, db_data)
  return np.sum((datos_experimentales - datos_predichos)**2)

#5

def fit_models_to_data(aw_data, db_data, temperature, initial_guess):
    """
    Ajusta ambos modelos a un conjunto de datos y retorna los parámetros optimizados
    """
    resultados = {}

    initial_guess = [0.1, 0.1, 0.1, 0.1]

    # Ajustar modelo Peleg
    resultado_peleg = minimize(chi_cuadrada_peleg, initial_guess,
                          args=(aw_data, db_data),
                          method='Nelder-Mead')
    resultados['peleg'] = {
        'params': resultado_peleg.x,
        'chi2': resultado_peleg.fun
    }

    # Ajustar modelo DLP
    resultado_dpl = minimize(chi_cuadrada_dlp, initial_guess,
                        args=(aw_data, db_data),
                        method='Nelder-Mead')
    resultados['dlp'] = {
        'params': resultado_dpl.x,
        'chi2': resultado_dpl.fun
    }

    return resultados


#6

#Ejecución del código

coffee_data_extracted = extract_all_coffee_data(df)

#Datos a 25°C
aw_data_25 = coffee_data_extracted['whole_beans'][25]['aw']
db_data_25 = coffee_data_extracted['whole_beans'][25]['db']

initial_guess = [0.1, 0.1, 0.1, 0.1]

resultados_25 = fit_models_to_data(aw_data_25, db_data_25, 25, initial_guess)

#Datos a 35°C
aw_data_35 = coffee_data_extracted['whole_beans'][35]['aw']
db_data_35 = coffee_data_extracted['whole_beans'][35]['db']

initial_guess = [0.1, 0.1, 0.1, 0.1]

resultados_35 = fit_models_to_data(aw_data_35, db_data_35, 35, initial_guess)

#Datos a 45°C
aw_data_45 = coffee_data_extracted['whole_beans'][45]['aw']
db_data_45 = coffee_data_extracted['whole_beans'][45]['db']

initial_guess = [0.1, 0.1, 0.1, 0.1]

resultados_45 = fit_models_to_data(aw_data_45, db_data_45, 45, initial_guess)

display (resultados_25)
display (resultados_35)
display (resultados_45)

{'peleg': {'params': array([2.43255203e+00, 8.35535600e-02, 3.76202684e+02, 3.56654977e+01]),
  'chi2': np.float64(4.668923719829734)},
 'dlp': {'params': array([ 3.00870145,  0.59326283, -1.89922437, -1.45301951]),
  'chi2': np.float64(22.710630512967015)}}

{'peleg': {'params': array([ 2.38467721,  0.03668128, 28.07476616, 12.88119407]),
  'chi2': np.float64(0.10547012188663593)},
 'dlp': {'params': array([ 2.43213008,  0.36699407, -0.31194773, -0.845447  ]),
  'chi2': np.float64(0.37499089691537096)}}

{'peleg': {'params': array([ 2.39006752,  0.06276725, 25.84963797, 10.14494933]),
  'chi2': np.float64(0.1472404812550843)},
 'dlp': {'params': array([ 2.15023762,  0.16317404,  0.31656039, -0.71528658]),
  'chi2': np.float64(0.4246196620328123)}}

Analizando los resultados, determinamos que para las 3 temperaturas dadas, el valor de chi^2  siempre es menor con el método peleg.

Esto nos indica que, de acuerdo al método aplicado en el código, el método peleg es más efectivo que el dlp.

Comparando con los resultados reportados en el artículo (página 5 - 6), los ajustes de R^2 son mayores para el método peleg, lo cual nos hace llegar a la misma conclusión.

EL MODELO PELEG DESCRIBE MEJOR LOS RESULTADOS QUE EL MODELO DLP, EN BASE A QUE EL VALOR DE CHI^2 FUE MENOR.